In [1]:
!kubectl get nodes gke-two-node-cluster-node-pool-1-6f09704a-8bcl --show-labels

NAME                                             STATUS   ROLES    AGE    VERSION               LABELS
gke-two-node-cluster-node-pool-1-6f09704a-8bcl   Ready    <none>   122m   v1.33.2-gke.1240000   beta.kubernetes.io/arch=amd64,beta.kubernetes.io/instance-type=e2-standard-4,beta.kubernetes.io/os=linux,cloud.google.com/gke-boot-disk=pd-standard,cloud.google.com/gke-container-runtime=containerd,cloud.google.com/gke-cpu-scaling-level=4,cloud.google.com/gke-logging-variant=DEFAULT,cloud.google.com/gke-max-pods-per-node=110,cloud.google.com/gke-memory-gb-scaling-level=16,cloud.google.com/gke-nodepool=node-pool-1,cloud.google.com/gke-os-distribution=cos,cloud.google.com/gke-provisioning=standard,cloud.google.com/gke-stack-type=IPV4,cloud.google.com/machine-family=e2,cloud.google.com/private-node=false,failure-domain.beta.kubernetes.io/region=us-central1,failure-domain.beta.kubernetes.io/zone=us-central1-a,kubernetes.io/arch=amd64,kubernetes.io/hostname=gke-two-node-cluster-node-pool-1-6f097

In [ ]:
!gcloud compute disks create api-update-disk --size=5Gi --zone=us-central1-a

In [ ]:
!gcloud compute disks create api-inference-disk --size=5Gi --zone=us-central1-a

In [ ]:
!gcloud compute disks create mlflow-disk --size=5Gi --zone=us-central1-a


In [ ]:
# !kubectl apply -f deploy_real_kubernetes/deploy_micro_inf.yaml
!kubectl apply -f deploy_real_kubernetes/deploy_micro_up.yaml

In [ ]:
# only get 10 latest events. In windows. check mlflow. tail in windows
!kubectl get events | findstr /R "upd" 

In [99]:
# a = !kubectl get pods | findstr /R "mlflow"
a = !kubectl get pods
a

['NAME                            READY   STATUS    RESTARTS   AGE',
 'api-inferencia-54c6bc6b-7bklx   1/1     Running   0          2m15s',
 'api-update-5b8d584b6f-hmt8q     1/1     Running   0          44m',
 'mlflow-546948fc94-b5khf         1/1     Running   0          91m']

In [52]:
# api-update-5b8d584b6f-q2mc5 see logs
!kubectl logs api-update-5b8d584b6f-q2mc5

Error from server (BadRequest): container "api-update" in pod "api-update-5b8d584b6f-q2mc5" is waiting to start: ContainerCreating


In [98]:
!kubectl describe pod {a[1].split()[0]}

Name:             api-inferencia-54c6bc6b-7bklx
Namespace:        default
Priority:         0
Service Account:  default
Node:             gke-two-node-cluster-node-pool-1-6f09704a-21b4/10.10.0.6
Start Time:       Sat, 09 Aug 2025 13:15:56 +0700
Labels:           app=api-inferencia
                  pod-template-hash=54c6bc6b
Annotations:      <none>
Status:           Running
IP:               10.168.2.12
IPs:
  IP:           10.168.2.12
Controlled By:  ReplicaSet/api-inferencia-54c6bc6b
Containers:
  api-inferencia:
    Container ID:  containerd://55792c14cbf97d705bce9e45c3f77b904e7a48cfdada7332d2ecd7263905a2c5
    Image:         mgrodrigues001/api_flask_inferencia:inf_v8.0
    Image ID:      docker.io/mgrodrigues001/api_flask_inferencia@sha256:e4e7b098684450a61bba0eb36086cbb107d6ede30021e6b9caeb0faea20fbcf6
    Port:          5001/TCP
    Host Port:     0/TCP
    Command:
      /bin/sh
    Args:
      -c
      tail -f /dev/null
    State:          Running
      Started:      Sat, 09 A

In [93]:
!kubectl get pvc

NAME                 STATUS    VOLUME                                     CAPACITY   ACCESS MODES   STORAGECLASS   VOLUMEATTRIBUTESCLASS   AGE
api-inferencia-pvc   Pending                                                                        standard-rwo   <unset>                 18s
api-update-pvc       Bound     pvc-c85ddeff-6037-4692-a8c3-55f350e44f50   5Gi        RWO            standard       <unset>                 41m
mlflow-pvc           Bound     pvc-a6ef3a45-7390-47c7-a101-6a8ecfa60292   5Gi        RWO            standard       <unset>                 88m


In [92]:
!kubectl get pv

NAME                                       CAPACITY   ACCESS MODES   RECLAIM POLICY   STATUS      CLAIM                    STORAGECLASS   VOLUMEATTRIBUTESCLASS   REASON   AGE
api-inference-pv                           5Gi        RWO            Retain           Available                                           <unset>                          2m18s
api-inferencia-pv                          5Gi        RWO            Retain           Available                                           <unset>                          8s
api-update-pv                              5Gi        RWO            Retain           Available                                           <unset>                          41m
mlflow-pv                                  5Gi        RWO            Retain           Available                                           <unset>                          88m
pvc-a6ef3a45-7390-47c7-a101-6a8ecfa60292   5Gi        RWO            Delete           Bound       default/mlflow-pvc       s

In [35]:
!kubectl delete pod {a[0].split()[0]}
# !kubectl delete pod {a[1].split()[0]}

pod "mlflow-546948fc94-bnhnl" deleted


In [37]:
a

['mlflow-546948fc94-wvnzk       0/1     ContainerCreating   0          7s',
 'mlflow-6cbdd998d8-4fj5l       0/1     Pending             0          12s']

In [5]:
!kubectl get nodes

NAME                                             STATUS   ROLES    AGE   VERSION
gke-two-node-cluster-node-pool-1-6f09704a-21b4   Ready    <none>   29m   v1.33.2-gke.1240000
gke-two-node-cluster-node-pool-1-6f09704a-seg5   Ready    <none>   20m   v1.33.2-gke.1240000


In [95]:
# how to unapply a file
# !kubectl apply -f deploy_real_kubernetes/04-mlflow.yaml
# !kubectl delete -f deploy_real_kubernetes/04-mlflow.yaml

# !kubectl apply -f deploy_real_kubernetes/deploy_micro_up.yaml
# !kubectl delete -f deploy_real_kubernetes/deploy_micro_up.yaml

!kubectl apply -f deploy_real_kubernetes/deploy_micro_inf.yaml
# !kubectl delete -f deploy_real_kubernetes/deploy_micro_inf.yaml

deployment.apps/api-inferencia created
persistentvolumeclaim/api-inferencia-pvc created
persistentvolume/api-inferencia-pv created


In [81]:
!kubectl events | findstr /R "inf"

15m (x26 over 153m)      Warning   FailedCreate                 ReplicaSet/api-inferencia-6768dc5ffb                  Error creating: pods "api-inferencia-6768dc5ffb-" is forbidden: error looking up service account default/my-app-sa: serviceaccount "my-app-sa" not found
3m53s (x601 over 153m)   Normal    WaitForFirstConsumer         PersistentVolumeClaim/api-inferencia-pvc              waiting for first consumer to be created before binding
2m8s                     Normal    ScalingReplicaSet            Deployment/api-inferencia                             Scaled up replica set api-inferencia-9d54bb6b6 from 0 to 1
118s (x12 over 2m8s)     Warning   FailedCreate                 ReplicaSet/api-inferencia-9d54bb6b6                   Error creating: pods "api-inferencia-9d54bb6b6-" is forbidden: error looking up service account default/my-app-sa: serviceaccount "my-app-sa" not found
112s (x3 over 2m8s)      Normal    WaitForFirstConsumer         PersistentVolumeClaim/api-inferencia-pvc    

In [84]:
!kubectl get pods 

NAME                            READY   STATUS              RESTARTS   AGE
api-inferencia-54c6bc6b-ctxf5   0/1     ContainerCreating   0          15s
api-update-5b8d584b6f-hmt8q     1/1     Running             0          39m
mlflow-546948fc94-b5khf         1/1     Running             0          86m


In [14]:
!kubectl delete pod mlflow-6cbdd998d8-gcdqv

Error from server (NotFound): pods "mlflow-6cbdd998d8-gcdqv" not found


In [59]:
!kubectl get pvc

NAME                 STATUS    VOLUME                                     CAPACITY   ACCESS MODES   STORAGECLASS   VOLUMEATTRIBUTESCLASS   AGE
api-inferencia-pvc   Pending                                                                        standard-rwo   <unset>                 172m
mlflow-pvc           Bound     pvc-a6ef3a45-7390-47c7-a101-6a8ecfa60292   5Gi        RWO            standard       <unset>                 41m


In [125]:
a = !kubectl get pods | findstr /R "upd"
a

['api-update-5b8d584b6f-hmt8q     1/1     Running   0          97m']

In [4]:
!kubectl apply -f deploy_real_kubernetes/04-mlflow.yaml

persistentvolumeclaim/mlflow-pvc unchanged
service/mlflow unchanged


error: error parsing deploy_real_kubernetes/04-mlflow.yaml: invalid Yaml document separator: null


In [148]:
# do ls inside a.split()[0]
# !kubectl exec {a[0].split()[0]} -- ls /app/

# get ip address
# !kubectl exec {a[0].split()[0]} -- hostname -i

# copy API_inferencia_V8.0.py to ../../temp
# a = !kubectl get pods | findstr /R "upd"
# !kubectl cp ../temp/upd/API_update_V8.1.py {a[0].split()[0]}:/app/API_update_V8.1.py 
# !kubectl cp ../../Datasets/real_usage/AGR_a_first_train.csv {a[0].split()[0]}:/app/AGR_a_first_train.csv
# !kubectl cp ../../Datasets/real_usage/AGR_g_first_train.csv {a[0].split()[0]}:/app/AGR_g_first_train.csv

a = !kubectl get pods | findstr /R "inf"
!kubectl cp ../temp/inf/API_inferencia_V8.0.py {a[0].split()[0]}:/app/API_inferencia_V8.0.py


In [ ]:
!kubectl exec {a[0].split()[0]} -- python3 /app/API_inferencia_V8.0.py

In [129]:
a = !kubectl get pods | findstr /R "inf"
# !kubectl exec {a[0].split()[0]} -- rm AGR_a_first_train.csv
# !kubectl exec {a[0].split()[0]} -- rm AGR_g_first_train.csv
# !kubectl exec {a[0].split()[0]} -- rm API_update_V8.1.py
!kubectl exec {a[0].split()[0]} -- ls

API_inferencia_V8.0.py
api_inference_10.168.2.12.log
lost+found


In [ ]:
!kubectl get pods --all-namespaces

In [132]:
# check public ip address of the kubernetes GKE
!kubectl get services --all-namespaces

NAMESPACE     NAME                   TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)            AGE
default       api-inferencia         NodePort    34.118.235.188   <none>        5001:32001/TCP     4h40m
default       api-update             NodePort    34.118.233.94    <none>        5002:32002/TCP     4h40m
default       kubernetes             ClusterIP   34.118.224.1     <none>        443/TCP            5h37m
default       mlflow                 NodePort    34.118.238.242   <none>        5000:30500/TCP     149m
gmp-system    alertmanager           ClusterIP   None             <none>        9093/TCP           5h35m
gmp-system    gmp-operator           ClusterIP   34.118.235.70    <none>        8443/TCP,443/TCP   5h35m
gmp-system    rule-evaluator         ClusterIP   34.118.238.3     <none>        19092/TCP          5h35m
kafka         kafka-service          NodePort    34.118.235.72    <none>        9092:32092/TCP     4h40m
kafka         zookeeper-service      NodePort    34.118.23

In [133]:
!gcloud container clusters describe two-node-cluster --region us-central1-a --format="value(privateClusterConfig.publicEndpoint)"


34.172.244.58


In [ ]:
!kubectl describe service api-inferencia

In [141]:
# scale it to 3 service api-inferencia
!kubectl scale deployment api-inferencia --replicas=3


deployment.apps/api-inferencia scaled


In [143]:
a = !kubectl get pods | findstr /R "inf"
a

['api-inferencia-54c6bc6b-7bklx   1/1     Running   0          3h32m',
 'api-inferencia-54c6bc6b-c4979   1/1     Running   0          42s',
 'api-inferencia-54c6bc6b-zsjcx   1/1     Running   0          42s']

In [146]:
# check ls of a[0].split()[0]
!kubectl exec {a[0].split()[0]} -- ls
!kubectl exec {a[1].split()[0]} -- ls

API_inferencia_V8.0.py
api_inference_10.168.2.12.log
load_model_10.168.2.12.csv
lost+found
API_inferencia_V8.0.py
api_inference_10.168.2.12.log
load_model_10.168.2.12.csv
lost+found


In [136]:
!kubectl get ingress

No resources found in default namespace.


In [ ]:
!curl http://34.172.244.58:30500

In [139]:
!kubectl get nodes -o wide


NAME                                             STATUS   ROLES    AGE     VERSION               INTERNAL-IP   EXTERNAL-IP     OS-IMAGE                             KERNEL-VERSION   CONTAINER-RUNTIME
gke-two-node-cluster-node-pool-1-6f09704a-21b4   Ready    <none>   4h6m    v1.33.2-gke.1240000   10.10.0.6     34.60.133.175   Container-Optimized OS from Google   6.6.93+          containerd://2.0.4
gke-two-node-cluster-node-pool-1-6f09704a-seg5   Ready    <none>   3h57m   v1.33.2-gke.1240000   10.10.0.7     34.58.44.149    Container-Optimized OS from Google   6.6.93+          containerd://2.0.4


In [140]:
!curl http://34.60.133.175:30500

<!doctype html><html lang="en"><head><meta charset="utf-8"/><meta name="viewport" content="width=device-width,initial-scale=1,shrink-to-fit=no"/><link rel="shortcut icon" href="./static-files/favicon.ico"/><meta name="theme-color" content="#000000"/><link rel="manifest" href="./static-files/manifest.json"/><title>MLflow</title><script defer="defer" src="static-files/static/js/main.6125589f.js"></script><link href="static-files/static/css/main.3b6f4584.css" rel="stylesheet"></head><body><noscript>You need to enable JavaScript to run this app.</noscript><div id="root"></div><div id="modal"></div></body></html>


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   615  100   615    0     0   1337      0 --:--:-- --:--:-- --:--:--  1339


In [1]:
import os
import threading


def run_script(pod_name, script_path, i=1):
    os.system(f"kubectl exec {pod_name} -- nohup python3 {script_path} > output{i}.log 2>&1 &")
    # !kubectl exec {pod_name} -- nohup python3 /app/API_inferencia_V8.0.py &



In [ ]:

a = !kubectl get pods | findstr /R "inf"
for i in range(len(a)):
    print(f"Pod {i+1}: {a[i].split()[0]}")
    # use nohup and threading to run the script in the background

    # threads = []
    pod_name = a[i].split()[0]
    t = threading.Thread(target=run_script, args=(pod_name,"/app/API_inferencia_V8.0.py", i+1))
    # threads.append(t)
    t.start()


In [36]:

a = !kubectl get pods | findstr /R "upd"
t = threading.Thread(target=run_script, args=(a[0].split()[0], "/app/API_update_V8.1.py"))
t.start()



In [3]:
a = !kubectl get pods | findstr /R "upd"
run_script(a[0].split()[0], "/app/API_update_V8.1.py")

In [10]:
a = !kubectl get pods | findstr /R "inf"


In [ ]:

a = !kubectl get pods | findstr /R "inf"
# # see python processes running in the pod
print("Checking Python processes in pods:")
print(f"Pod 1: {a[0].split()[0]}")
!kubectl exec {a[0].split()[0]} -- ps aux | findstr /R "python"
print(f"Pod 2: {a[1].split()[0]}")
!kubectl exec {a[1].split()[0]} -- ps aux | findstr /R "python"
print(f"Pod 3: {a[2].split()[0]}")
!kubectl exec {a[2].split()[0]} -- ps aux | findstr /R "python"

# # kill the python process
# !kubectl exec {a[0].split()[0]} -- pkill -f API
# !kubectl exec {a[1].split()[0]} -- pkill -f API
# !kubectl exec {a[2].split()[0]} -- pkill -f API

Checking Python processes in pods:
Pod 1: api-inferencia-54c6bc6b-7bklx
root        1792  0.2  0.3 909516 129804 ?       Sl   10:53   0:06 /usr/local/bin/python3 /app/API_inferencia_V8.0.py
Pod 2: api-inferencia-54c6bc6b-c4979
root         247  0.3  0.3 909512 130108 ?       Sl   10:53   0:06 /usr/local/bin/python3 /app/API_inferencia_V8.0.py
Pod 3: api-inferencia-54c6bc6b-zsjcx
root         265  0.2  0.3 909516 129816 ?       Sl   10:53   0:06 /usr/local/bin/python3 /app/API_inferencia_V8.0.py


In [7]:
import os
import threading

b = !kubectl get pods | findstr /R "upd"

def run_update_script():
    os.system(f"kubectl exec {b[0].split()[0]} -- python3 /app/API_update_V8.1.py")
t = threading.Thread(target=run_update_script)
t.start()


In [3]:
b = !kubectl get pods | findstr /R "upd"
b

['api-update-5b8d584b6f-hmt8q     1/1     Running   0          5h54m']

In [8]:
!kubectl exec {b[0].split()[0]} -- ps aux | findstr /R "python"

root       25390 72.4  0.4 959728 161772 ?       Ssl  11:28   0:01 python3 /app/API_update_V8.1.py
root       25411  110  0.3 562708 101748 ?       Rl   11:28   0:00 /usr/local/bin/python3 /app/API_update_V8.1.py


In [5]:
# kill the python process
!kubectl exec {b[0].split()[0]} -- pkill -f API_update

In [ ]:
# Run the scripts

import os
import threading

update_pods = !kubectl get pods | findstr /R "upd"

def run_update_script(pod_name):
    os.system(f"kubectl exec {pod_name} -- nohup python3 /app/API_update_V8.1.py > {pod_name}_output.log 2>&1 &")
t = threading.Thread(target=run_update_script, args=(update_pods[0].split()[0],))
t.start()

def run_inference_script(pod_name, script_path):
    os.system(f"kubectl exec {pod_name} -- nohup python3 {script_path} > {pod_name}_output.log 2>&1 &")

inference_pods = !kubectl get pods | findstr /R "inf"
for pod in inference_pods:
    pod_name = pod.split()[0]
    script_path = "/app/API_inferencia_V8.0.py"
    t = threading.Thread(target=run_inference_script, args=(pod_name, script_path))
    t.start()



In [12]:
for pod in inference_pods:
    !kubectl exec {pod.split()[0]} -- pkill -f API
for pod in update_pods:
    !kubectl exec {pod.split()[0]} -- pkill -f API

In [ ]:

# # see python processes running in the pod
print("Checking Python processes in pods:")
for pod in inference_pods:
    print(f"Pod: {pod.split()[0]}")
    !kubectl exec {pod.split()[0]} -- ps aux | findstr /R "python"

for pod in update_pods:
    print(f"Pod: {pod.split()[0]}")
    !kubectl exec {pod.split()[0]} -- ps aux | findstr /R "python"

Checking Python processes in pods:
Pod: api-inferencia-57f4c5dcc5-5www7
Pod: api-inferencia-57f4c5dcc5-d4dgq
Pod: api-inferencia-57f4c5dcc5-ks2vn
Pod: api-inferencia-57f4c5dcc5-w94hw
Pod: api-update-6df9ff788-v5fvb


In [27]:
# Copy everything from C:\Users\adilm\OneDrive\Documents\large-scale-online-learning\Datasets\real_usage
# to container with id d87523e7d using docker cp since it is in local Windows environment
# create a directory in the container if it does not exist
!docker exec d87523e7d mkdir -p /app/large-scale-online-learning/MLOps-Architecture/Serialization_Datasets
# !docker exec d87523e7d rm -r /app/Datasets/real_usage
# !docker exec d87523e7d ls /app/Datasets/real_usage
# !docker cp C:\Users\adilm\OneDrive\Documents\large-scale-online-learning\MLOps-Architecture\Serialization_Datasets\testing.ipynb d87523e7d:/app/large-scale-online-learning/MLOps-Architecture/Serialization_Datasets/
!docker cp C:\Users\adilm\OneDrive\Documents\large-scale-online-learning\MLOps-Architecture\Serialization_Datasets\locust-testing.py d87523e7d:/app/large-scale-online-learning/MLOps-Architecture/Serialization_Datasets/

In [76]:
!docker commit d87523e7d auliadil/load-testing-rodrigues:v1

sha256:6fcf2f8fc892644435fc2ab38a9adf2d0011ed87cdb34a8facc46dae8e7566bb


In [32]:
!docker cp "C:\Users\adilm\.kube" d87523e7d:/root/.kube

In [33]:
!kubectl create serviceaccount superadmin-sa -n kafka

serviceaccount/superadmin-sa created


In [34]:
!kubectl create clusterrolebinding superadmin-binding --clusterrole=cluster-admin --serviceaccount=kafka:superadmin-sa

clusterrolebinding.rbac.authorization.k8s.io/superadmin-binding created


In [35]:
!kubectl create token superadmin-sa -n kafka --duration=24h


eyJhbGciOiJSUzI1NiIsImtpZCI6IlNrZUFqSUhOZUc1Ynl6YUJtWTRJTndialFnNFFrdzl4eEpUWTFORHhuY2cifQ.eyJhdWQiOlsiaHR0cHM6Ly9jb250YWluZXIuZ29vZ2xlYXBpcy5jb20vdjEvcHJvamVjdHMvZmFzdC1sZWFybmVyLXByb2plY3QvbG9jYXRpb25zL3VzLWNlbnRyYWwxLWEvY2x1c3RlcnMvdHdvLW5vZGUtY2x1c3RlciJdLCJleHAiOjE3NTQ5MDM5MjcsImlhdCI6MTc1NDgxNzUyNywiaXNzIjoiaHR0cHM6Ly9jb250YWluZXIuZ29vZ2xlYXBpcy5jb20vdjEvcHJvamVjdHMvZmFzdC1sZWFybmVyLXByb2plY3QvbG9jYXRpb25zL3VzLWNlbnRyYWwxLWEvY2x1c3RlcnMvdHdvLW5vZGUtY2x1c3RlciIsImp0aSI6ImI1NWFiMDNjLTE5ZWEtNDBhZS1hNTBiLWJmOGRiZDgwMDhkNSIsImt1YmVybmV0ZXMuaW8iOnsibmFtZXNwYWNlIjoia2Fma2EiLCJzZXJ2aWNlYWNjb3VudCI6eyJuYW1lIjoic3VwZXJhZG1pbi1zYSIsInVpZCI6IjU0YzI0YjgyLWMyNGYtNGRkNC1hYjZkLTY1NTMzZDI5Y2IzZCJ9fSwibmJmIjoxNzU0ODE3NTI3LCJzdWIiOiJzeXN0ZW06c2VydmljZWFjY291bnQ6a2Fma2E6c3VwZXJhZG1pbi1zYSJ9.bzsd-VjRZ_Ob41oa_KMkXElYF3TSK17lJlj4KVPK4hGFxCfRAmQUL-FU2W-fPinwbPA0RVDKtD1zrcoYQvdYc1lXE1QtNIMawyYJwnzTXwl0AcLAzXGfUTCXXlWJpIDX6agcIAPW8zsga4Su0Eq-d9UH61Trmh0KEP0wlaFNmVz3CUWiFGg_lpot3xIl-4Kdw7Y-nPI_Afm6SRxUTrjR

In [41]:
# !dir "C:\Users\adilm\AppData\Roaming\gcloud"
!docker exec d87523e7d rm -r /root/.config/gcloud
!docker cp "C:\Users\adilm\AppData\Roaming\gcloud" d87523e7d:/root/.config


rm: cannot remove '/root/.config/gcloud': No such file or directory


In [67]:
# !gcloud compute ssh load-testing-instance --zone=us-central1-a --command="ls /etc/docker"
# !gcloud compute ssh load-testing-instance --zone=us-central1-a --command="sudo docker ps"
# a = '{"live-restore": true, \"log-opts\": {\"tag\': \"{{.Name}}\"}, \"storage-driver\": \"overlay2\", \"mtu\": 1460, \"hosts\": [\"unix:///var/run/docker.sock\", \"tcp://0.0.0.0:2375\"]}'
a = '{"live-restore": true, "log-opts": {"tag": "{{.Name}}"}, "storage-driver": "overlay2", "mtu": 1460, "hosts": ["unix:///var/run/docker.sock", "tcp://0.0.0.0:2375"]}'

# write a to /etc/docker/daemon.json
# !gcloud compute ssh standalone-load-tester --zone=us-central1-a --command="echo jojo" 

# write the contents of variable a to /etc/docker/daemon.json using sudo
!gcloud compute ssh standalone-load-tester --zone=us-central1-a --command="echo '{a}' | sudo tee /etc/docker/daemon.json"
!gcloud compute ssh standalone-load-tester --zone=us-central1-a --command="cat /etc/docker/daemon.json" 

{live-restore: true, log-opts: {tag: {{.Name}}}, storage-driver: overlay2, mtu: 1460, hosts: [unix:///var/run/docker.sock, tcp://0.0.0.0:2375]}
{live-restore: true, log-opts: {tag: {{.Name}}}, storage-driver: overlay2, mtu: 1460, hosts: [unix:///var/run/docker.sock, tcp://0.0.0.0:2375]}


In [87]:
# !gcloud compute ssh standalone-load-tester --zone=us-central1-a --command="sudo docker ps"
#sudo systemctl restart docker
# !gcloud compute ssh standalone-load-tester --zone=us-central1-a --command="systemctl restart docker"
# !gcloud compute ssh standalone-load-tester --zone=us-central1-a --command="whoami"
# add this in docker group adilmuhammad706_gmail_com
# !gcloud compute ssh standalone-load-tester --zone=us-central1-a --command="docker ps"
# !gcloud compute instances stop standalone-load-tester --zone=us-central1-a
# !gcloud compute instances start standalone-load-tester --zone=us-central1-a
# gcloud_template = "gcloud compute ssh standalone-load-tester --zone=us-central1-a"
# tail dev null
gcloud_template = "gcloud compute ssh standalone-load-tester --zone=us-central1-a"
# docker_command = "sudo docker run -d --name load-testing-container auliadil/load-testing-rodrigues:v1 tail -f /dev/null"
docker_command = "sudo ls /"
!{gcloud_template} --command="{docker_command}"

bin
boot
dev
etc
home
lib
lib64
lost+found
mnt
opt
postinst
proc
root
run
sbin
sys
tmp
usr
var


In [ ]:
def copy_to_load_testing_docker(file_path, destination_path="/app/large-scale-online-learning/MLOps-Architecture/Serialization_Datasets/"):
    """Copy a file to the load testing Docker container."""
    filename = file_path.split("\\")[-1]  # Get the filename from the path
    gcloud_template_scp_local_to_remote = f"gcloud compute scp --zone=us-central1-a {file_path} standalone-load-tester:/{filename}"
    # Ensure the destination directory exists in the container
    !{gcloud_template_scp_local_to_remote}
    gcloud_command = f"gcloud compute ssh standalone-load-tester --zone=us-central1-a --command='sudo docker cp /test/{filename} load-testing-container:{destination_path}'"
    !{gcloud_command}

copy_to_load_testing_docker(r"C:\Users\adilm\OneDrive\Documents\large-scale-online-learning\MLOps-Architecture\terraform_stuff\etc\daemon.json", "/app")

In [ ]:
# Generate SSH key pair on Windows (using Git Bash, WSL, or PowerShell)
# Run this command in your terminal:
!ssh-keygen -t rsa -b 4096 -f %USERPROFILE%\.ssh\id_rsa -C "adilmuhammad706_gmail_com"
# !echo 
# This creates:
# - Private key: ~/.ssh/id_rsa
# - Public key: ~/.ssh/id_rsa.pub

# To copy the public key to GCP project metadata:
# 1. Open ~/.ssh/id_rsa.pub and copy its contents.
# 2. Go to Google Cloud Console > Compute Engine > Metadata > SSH Keys.
# 3. Click "Edit" and paste the public key into the SSH Keys section.
# 4. Save.

# Now you can SSH from Windows to your Ubuntu VM on GCP using your private key.

^C


In [106]:
# !dir %USERPROFILE%\.ssh\
# !type %USERPROFILE%\.ssh\remote-ssh-demo-new
!type %USERPROFILE%\.ssh\id_rsa.pub

ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQDiLKTTdc5zYsKDR8D/TkzlHGG1XWc4wY8+Pxw+ooHb0jZghDYizteEsVU/kkUNcL7nGRKPlqgD/8oAD63/iifAah2cYFdJUv1RiK6wTLGpVyoH1d0fiNnn645pXlL56FFc8olp2aMBpzNO+tAJJ5ymQ1Y6TwzTzoahgjfBYEM5LEnEDYKVDPJoFABLPYXwRm0oIYa8N/K+0gO9r5NnCoefDyKXj6+B/YhbJWGjx6NH/6Qg5kiTdYAUjZMAAluUnx3M1yh6j+fQM6LzdJZRmvkoYEb7Ad6yndX61bn40553GylT2c1VoNLI7lXKjpVDHNJNJsVDFily4JcXmHy+VBif7HnPFih/q0KjK4yJCON4YDwQX7dU4qXBDi1GcfkLwpvbzK0dPZVsk1MQeT/5lqJ4pPZsvthMF7z60Eb9bTgmoxYycNGmREy5hi+rj/5UuRScea7u8uxeEZEsU5vQ5vFJqpWXGm4rYEpR4juydE8R57HPQ9aS1DA5IdM0r7VGuo0B1mVrWjKPwjT7d1mU3CftmZuTpdjylv1MxFOeepOdM384LmoOrlCRyfyrOHqedc9EV8ac1r+l51r3iSkAYvnxzhdnNlda1ZqdEA02tG+V8z7QYoLZJKBsPAOZ3gnsoY3zLBVG0411DN8Q1oJVejjU2uVe6EzEAjg1aRmk9ZoW9Q== adilmuhammad706@gmail.com
